In [3]:
#import libraries
import time
import pandas as pd
from google_play_scraper import reviews, Sort, app

In [5]:
APP_ID = "com.linkedin.android"       #LinkedIn app
LANG = "en"                           #set the language of reviews to English (universal)
COUNTRIES = ["us", "gb", "it", "fi", "kz"]   #set the countries to US, UK, Italy, Finland, Kazakhstan
MAX_REVIEWS_PER_COUNTRY = 2000        #set the maximum number of reviews per country

frames = []
for country in COUNTRIES:
    token = None
    rows = []
    while True:
        batch, token = reviews(
            APP_ID,
            lang = LANG,
            country = country,
            sort = Sort.NEWEST,
            count = 200,
            continuation_token = token
        )
        rows.extend(batch)
        print(f"{country}: {len(rows)}")
        if token is None or len(rows) >= MAX_REVIEWS_PER_COUNTRY:
            break
        time.sleep(0.8)
    
    if rows:
        df = pd.DataFrame(rows)
        df = df[["reviewId", "userName", "score", "at", "content", "replyContent"]] \
                .rename(columns = {"score":"rating", "at":"date", "content":"review", "replyContent":"dev_reply"})
        df["country"] = country
        df = df.drop_duplicates("reviewId")
        df = df[df["review"].str.strip().astype(bool)]
        frames.append(df)

us: 200
us: 400
us: 600
us: 800
us: 1000
us: 1200
us: 1400
us: 1600
us: 1800
us: 2000
gb: 200
gb: 400
gb: 600
gb: 800
gb: 1000
gb: 1200
gb: 1400
gb: 1600
gb: 1800
gb: 2000
it: 200
it: 400
it: 600
it: 800
it: 1000
it: 1200
it: 1400
it: 1600
it: 1800
it: 2000
fi: 200
fi: 400
fi: 600
fi: 800
fi: 1000
fi: 1200
fi: 1400
fi: 1600
fi: 1800
fi: 2000
kz: 200
kz: 400
kz: 600
kz: 800
kz: 1000
kz: 1200
kz: 1400
kz: 1600
kz: 1800
kz: 2000


In [7]:
#combine all
all_reviews = pd.concat(frames, ignore_index = True)
all_reviews.to_csv("linkedin_reviews_raw.csv", index = False)
print(all_reviews.groupby("country").size())
all_reviews.head()

country
fi    2000
gb    2000
it    2000
kz    2000
us    2000
dtype: int64


,reviewId,userName,rating,date,review,dev_reply,country
0,59436278-9150-462b-bee3-4def6655f057,Welbor,5,2025-09-14 20:33:00,wow amazing,None,us
1,b10e0798-4d6f-43d4-9b16-abdbd7c27a36,Pavan Kondapuram,5,2025-09-14 20:32:55,Very professional.,None,us
2,236f64d3-a1d4-40eb-b225-e403dec60daf,mustapha adamu,5,2025-09-14 20:13:54,excellent in making,None,us
3,2d77114d-b71a-4ff8-81ff-20cfab7cdfdc,Simoné B,5,2025-09-14 20:13:40,🌟🌟🌟🌟🌟,None,us
4,314e7f82-1b3c-4015-89bb-3ca15b308d29,Mamee Siraj,5,2025-09-14 20:02:13,All video network systems and mobile model SM-...,None,us
